<a href="https://colab.research.google.com/github/carlosarturo86/Coder_DataScienceII/blob/main/sqlite_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creación de Tablas y Conexión


In [144]:
import sqlite3

# Conectar a la base de datos (si no existe, se crea)
conn = sqlite3.connect('mi_base_de_datos.db')
cursor = conn.cursor()

'''
Se eliminan todas las tablas antes de su ejecución, dado que las BD son persistentes
 (datos se guardan en un archivo de disco, se quedan hasta que sean eliminados).
 Cada vez que se ejecuta un INSERT se van sumandos los resultados a los ya existentes.
'''
#Eliminar todas las tablas

cursor.execute('DROP TABLE IF EXISTS estudiantes')
cursor.execute('DROP TABLE IF EXISTS cursos')
cursor.execute('DROP TABLE IF EXISTS profesores')
cursor.execute('DROP TABLE IF EXISTS inscripciones')
cursor.execute('DROP TABLE IF EXISTS asignaciones')

# Crear la tabla estudiantes
cursor.execute('''
CREATE TABLE IF NOT EXISTS estudiantes (
    id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    edad INTEGER NOT NULL,
    promedio REAL
)
''')

# Crear la tabla cursos
cursor.execute('''
CREATE TABLE IF NOT EXISTS cursos (
    id INTEGER PRIMARY KEY ,
    nombre TEXT NOT NULL
)
''')

# Crear la tabla profesores
cursor.execute('''
CREATE TABLE IF NOT EXISTS profesores (
    id INTEGER PRIMARY KEY ,
    nombre TEXT NOT NULL,
    especialidad TEXT
)
''')

# Crear la tabla inscripciones
cursor.execute('''
CREATE TABLE IF NOT EXISTS inscripciones (
    id INTEGER PRIMARY KEY ,
    estudiante_id INTEGER,
    curso_id INTEGER,
    FOREIGN KEY(estudiante_id) REFERENCES estudiantes(id),
    FOREIGN KEY(curso_id) REFERENCES cursos(id)
)
''')

# Crear la tabla asignaciones
cursor.execute('''
CREATE TABLE IF NOT EXISTS asignaciones (
    id INTEGER PRIMARY KEY ,
    profesor_id INTEGER,
    curso_id INTEGER,
    FOREIGN KEY(profesor_id) REFERENCES profesores(id),
    FOREIGN KEY(curso_id) REFERENCES cursos(id)
)
''')

# Guardar los cambios
conn.commit()

## Insertar Datos

In [145]:
# Insertar datos en la tabla estudiantes
cursor.execute('''
INSERT INTO estudiantes (nombre, edad, promedio)
VALUES ('Juan', 15, 8.5),
       ('María', 14, 9.0),
       ('Pedro', 16, 7.8),
       ('Luis', 15, 8.0),
       ('Ana', 14, 9.2)
''')

# Insertar datos en la tabla cursos
cursor.execute('''
INSERT INTO cursos (nombre)
VALUES ('Matemáticas'),
       ('Historia'),
       ('Ciencias'),
       ('Literatura'),
       ('Física')
''')

# Insertar datos en la tabla profesores
cursor.execute('''
INSERT INTO profesores (nombre, especialidad)
VALUES ('Carlos', 'Matemáticas'),
       ('Laura', 'Historia'),
       ('Elena', 'Ciencias'),
       ('Javier', 'Literatura'),
       ('Sofía', 'Física')
''')

# Insertar datos en la tabla inscripciones
cursor.execute('''
INSERT INTO inscripciones (estudiante_id, curso_id)
VALUES (1, 1),
       (1, 2),
       (2, 3),
       (3, 4),
       (4, 5),
       (5, 1)
''')

# Insertar datos en la tabla asignaciones
cursor.execute('''
INSERT INTO asignaciones (profesor_id, curso_id)
VALUES (1, 1),
       (2, 2),
       (3, 3),
       (4, 4),
       (5, 5)
''')

# Guardar los cambios
conn.commit()

##Consultas sql

In [146]:
cursor.execute('SELECT * FROM inscripciones')
for row in cursor.fetchall():
  print(row)


(1, 1, 1)
(2, 1, 2)
(3, 2, 3)
(4, 3, 4)
(5, 4, 5)
(6, 5, 1)


In [147]:
# Consultar estudiantes con promedio mayor a 8
cursor.execute('SELECT * FROM estudiantes WHERE promedio > 8')
print("Estudiantes con promedio > 8:")
for row in cursor.fetchall():
    print(row)


Estudiantes con promedio > 8:
(1, 'Juan', 15, 8.5)
(2, 'María', 14, 9.0)
(5, 'Ana', 14, 9.2)


In [148]:
# Consultar estudiantes con un promedio no mayor a 8
cursor.execute('SELECT * FROM estudiantes WHERE NOT promedio > 8')
print("Estudiantes con promedio  <= 8:")
for row in cursor.fetchall():
    print(row)

Estudiantes con promedio  <= 8:
(3, 'Pedro', 16, 7.8)
(4, 'Luis', 15, 8.0)


In [149]:
# Consultar estudiantes con promedio mayor a 8 y edad mayor o igual a 15 años
cursor.execute('SELECT * FROM estudiantes WHERE promedio > 8 AND edad >= 15')
print("\nEstudiantes con promedio > 8 y edad >= 15:")
for row in cursor.fetchall():
    print(row)


Estudiantes con promedio > 8 y edad >= 15:
(1, 'Juan', 15, 8.5)


In [150]:
# INNER JOIN
cursor.execute('''
SELECT estudiantes.nombre AS estudiante, cursos.nombre AS curso
FROM estudiantes
INNER JOIN inscripciones ON estudiantes.id = inscripciones.estudiante_id
INNER JOIN cursos ON inscripciones.curso_id = cursos.id
''')
print("\nINNER JOIN:")
for row in cursor.fetchall():
    print(row)



INNER JOIN:
('Juan', 'Matemáticas')
('Juan', 'Historia')
('María', 'Ciencias')
('Pedro', 'Literatura')
('Luis', 'Física')
('Ana', 'Matemáticas')


In [151]:
# LEFT JOIN
cursor.execute('''
SELECT estudiantes.nombre AS estudiante, cursos.nombre AS curso
FROM estudiantes
LEFT JOIN inscripciones ON estudiantes.id = inscripciones.estudiante_id
LEFT JOIN cursos ON inscripciones.curso_id = cursos.id
''')
print("\nLEFT JOIN:")
for row in cursor.fetchall():
    print(row)


LEFT JOIN:
('Juan', 'Matemáticas')
('Juan', 'Historia')
('María', 'Ciencias')
('Pedro', 'Literatura')
('Luis', 'Física')
('Ana', 'Matemáticas')


In [152]:
# Insertar un nuevo estudiante y curso en una transacción
with conn:
    conn.execute('INSERT INTO estudiantes (nombre, edad, promedio) VALUES (?, ?, ?)', ('Marta', 15, 8.5))
    conn.execute('INSERT INTO inscripciones (estudiante_id, curso_id) VALUES (?, ?)', (6, 1))


cursor.execute('SELECT * FROM estudiantes ')
print("Nueva tabla Estudiantes")
cursor.fetchall()

Nueva tabla Estudiantes


[(1, 'Juan', 15, 8.5),
 (2, 'María', 14, 9.0),
 (3, 'Pedro', 16, 7.8),
 (4, 'Luis', 15, 8.0),
 (5, 'Ana', 14, 9.2),
 (6, 'Marta', 15, 8.5)]

In [153]:
# Agrupar por promedio y contar el número de estudiantes
cursor.execute('SELECT promedio, COUNT(*) AS cantidad FROM estudiantes GROUP BY promedio')
print("\nAgrupar por promedio y contar el número de estudiantes:")
for row in cursor.fetchall():
    print(row)


Agrupar por promedio y contar el número de estudiantes:
(7.8, 1)
(8.0, 1)
(8.5, 2)
(9.0, 1)
(9.2, 1)


In [154]:
# Agrupar por promedio y contar el número de estudiantes con promedio mayor a 8
cursor.execute('SELECT promedio, COUNT(*) AS cantidad FROM estudiantes GROUP BY promedio HAVING promedio > 8')
print("\nAgrupar por promedio y contar el número de estudiantes con promedio > 8:")
for row in cursor.fetchall():
    print(row)


Agrupar por promedio y contar el número de estudiantes con promedio > 8:
(8.5, 2)
(9.0, 1)
(9.2, 1)


##Consultas Usando Python - Pandas

In [155]:
import pandas as pd

# cargar la BD

df_estudiantes = pd.read_sql_query('SELECT * FROM estudiantes',conn)
df_estudiantes

,id,nombre,edad,promedio
0,1,Juan,15,8.5
1,2,María,14,9.0
2,3,Pedro,16,7.8
3,4,Luis,15,8.0
4,5,Ana,14,9.2
5,6,Marta,15,8.5


In [156]:
#Estudiantes con promedio mayor a 8 y edad mayor o igual a 15
print("\nEstudiantes con promedio > 8 y edad >= 15:")
df_estudiantes[(df_estudiantes['promedio']>8) & (df_estudiantes['edad'] >=15)]



Estudiantes con promedio > 8 y edad >= 15:


,id,nombre,edad,promedio
0,1,Juan,15,8.5
5,6,Marta,15,8.5


In [157]:
#Estudiantes que no tienen un promedio mayor a 8
df_estudiantes[~(df_estudiantes['promedio']>8)]

,id,nombre,edad,promedio
2,3,Pedro,16,7.8
3,4,Luis,15,8.0


In [158]:
# Agrupar por promedio y contar el número de estudiantes
print("\nAgrupar por promedio y contar el número de estudiantes:")
print(df_estudiantes.groupby('promedio').size())

# Agrupar por promedio y contar el número de estudiantes con promedio mayor a 8
print("\nAgrupar por promedio y contar el número de estudiantes con promedio > 8:")
print(df_estudiantes[df_estudiantes['promedio'] > 8].groupby('promedio').size())


Agrupar por promedio y contar el número de estudiantes:
promedio
7.8    1
8.0    1
8.5    2
9.0    1
9.2    1
dtype: int64

Agrupar por promedio y contar el número de estudiantes con promedio > 8:
promedio
8.5    2
9.0    1
9.2    1
dtype: int64


In [160]:
#Cerrar la conexión
conn.close()